In [1]:
import requests
import time
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
url_M = "http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyFullDown"
api_M = 'SOecclsdpfuvGmi+5d/vkz3M5XiRZEpLLLAm6oeRb4lI6TJSEsmP4nOvH6DfEznSzCw8o3+Ktedgq18jQIGUmg=='

In [3]:
url_H = 'http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncFullDown'
api_H = 'SOecclsdpfuvGmi+5d/vkz3M5XiRZEpLLLAm6oeRb4lI6TJSEsmP4nOvH6DfEznSzCw8o3+Ktedgq18jQIGUmg=='

In [5]:
def fetch_all_data(api_url, api_key, num_of_rows=100, delay=0.05):  # 50ms 대기
    """
    API에서 totalCount를 자동으로 가져와 전체 데이터를 수집하는 함수
    :param api_url: API의 기본 URL
    :param api_key: API 키
    :param num_of_rows: 한 번에 가져올 데이터 개수 (기본 100)
    :param delay: 요청 간 대기 시간 (초)
    :return: 전체 응답 리스트
    """
    # totalCount 가져오기
    params = {
        'ServiceKey': api_key,
        'pageNo': '1',
        'numOfRows': '1'  # 한 개만 요청하여 totalCount 확인
    }
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        total_count_elem = root.find(".//totalCount")
        total_count = int(total_count_elem.text) if total_count_elem is not None else 0
    else:
        print(f"totalCount 가져오기 실패: 상태 코드 {response.status_code}")
        return []
    
    all_responses = []
    total_pages = (total_count // num_of_rows) + (1 if total_count % num_of_rows else 0)

    for page in range(1, total_pages + 1):
        params = {
            'ServiceKey': api_key,
            'pageNo': str(page),
            'numOfRows': str(num_of_rows)
        }
        response = requests.get(api_url, params=params)

        if response.status_code == 200:
            all_responses.append(response.content)
        else:
            print(f"페이지 {page} 요청 실패: 상태 코드 {response.status_code}")

        time.sleep(delay)  # 50ms 대기 (30TPS 제한 맞추기)

    return all_responses

In [6]:
def convert_xml_to_dataframe(xml_data_list):
    """fetch_all_data()로 가져온 XML 데이터를 DataFrame으로 변환"""
    parsed_data = []

    for xml_data in xml_data_list:
        root = ET.fromstring(xml_data)

        for item in root.findall(".//item"):
            data = {child.tag: child.text for child in item}
            parsed_data.append(data)

    return pd.DataFrame(parsed_data) if parsed_data else None

In [7]:
response_M = fetch_all_data(url_M, api_M)

In [8]:
response_H = fetch_all_data(url_H, api_H)

In [27]:
df_H = convert_xml_to_dataframe(response_H)

In [18]:
df_M = convert_xml_to_dataframe(response_M)

In [22]:
df_M.to_excel("C:/Users/hk522/Desktop/약국 데이터.xlsx", index=False)